In [10]:
import pickle
import numpy as np
import pandas as pd

In [11]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [12]:
predicted_test = test.copy()

feature_1= 'x'
feature_2='y'

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models_1_2/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

#log_likelihoods = pd.DataFrame()
#log_likelihoods['predicted_class'] = classes
#log_like_max = []
#top_3_loglikelihood = []
top_3_predicted_classes = []
predicted_class = []
#is_predicted = []
true_label=[]

for i in range(len(test)):
    angle_1 = test[feature_1].iloc[i]
    angle_2 = test[feature_2].iloc[i]
    X=np.array([angle_1,angle_2]).T
    true_label.append(test['label'].iloc[i])
    list_log_likelihoods = []
    for model in models:
        
        list_log_likelihoods.append(model.score(X,lengths=len(X)))
        pass

    predicted_class.append(np.argmax(list_log_likelihoods))

    top_3_values = sorted(list_log_likelihoods)[-3:]
    top_3_predicted_classes.append([i for i, x in enumerate(list_log_likelihoods) if x == top_3_values[0] or x == top_3_values[1] or x == top_3_values[2]])
    
# model_0.score_samples(angles_0)

Degenerate mixture covariance
Degenerate mixture covariance
/home/thomas/anaconda3/envs/pml/lib/python3.9/site-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
/home/thomas/anaconda3/envs/pml/lib/python3.9/site-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
/home/thomas/anaconda3/envs/pml/lib/python3.9/site-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in log
  log_cur_weights = np.log(self.weights_[i_comp])
Degenerate mixture covariance
Degenerate mixture covariance
Degenerate mixture covariance
/home/thomas/anaconda3/envs/pml/lib/python3.9/site-packages/hmmlearn/hmm.py:887: RuntimeWarning: divide by zero encountered in 

In [13]:
print(predicted_class)
print(true_label)

print(len(predicted_class))
print(len(true_label))

[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
[4, 1, 9, 3, 5, 8, 0, 4, 6, 0, 8, 4, 7, 2, 6, 4, 7, 3, 1, 2, 8, 0, 9, 0, 5, 3, 6, 4, 5, 2, 4, 5, 2, 0, 0, 4, 2, 1, 2, 9, 3, 2, 9, 3, 1, 7, 5, 5, 8, 6, 7, 4, 5, 3, 3, 1, 0, 9, 6, 8, 9, 0, 6, 9, 9, 8, 1, 0, 2, 4, 5, 2, 6, 9, 3, 4, 4, 2, 6, 0, 9, 9, 8, 7, 2,

In [14]:
from sklearn.metrics.cluster import contingency_matrix 


In [15]:
matrix=contingency_matrix(true_label, predicted_class)

In [16]:
matrix

array([[34],
       [23],
       [23],
       [22],
       [25],
       [25],
       [25],
       [21],
       [24],
       [26]])

In [17]:
accuracy=np.sum(np.diag(matrix))/np.sum(matrix)
print("Accuracy: ", accuracy)

Accuracy:  0.13709677419354838


In [18]:
# Calculate top 3 accuracy
sum_top_3 = 0
for i in range(len(top_3_predicted_classes)):
    if true_label[i] in top_3_predicted_classes[i]:
        sum_top_3 += 1

accuracy_top_3 = sum_top_3/len(predicted_test["label"])
print("Accuracy top 3: ", accuracy_top_3)

Accuracy top 3:  0.7903225806451613
